# Kurvanpassningen med Pythons egna bibliotek

Först ska vi ladda lite externa bibliotek

In [ ]:
from matplotlib import pyplot as plt
import scipy.optimize as opt
import numpy as np
import math
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 9]

Sedan ange vi vår mätdata i källkoden i form av två listor: 

uppmätta spänningar $U_{meas}$ och uppmätta strömar $I_{meas}$ i variablerna `Umeas` och `Imeas`

In [ ]:
Umeas = [ 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, -1.0, -2.0, -3.0, -4.0, -5.0, -6.0, -7.0, -8.0, -9.0, -10.0]
Imeas = [ 0.0, 2.8, 5.8, 8.8, 11.7, 14.7, 17.7, 20.7, 23.7, 26.6, 29.6, -2.8, -5.8, -8.7, -11.7, -14.7, -17.7, -20.6, -23.6, -26.6, -29.6]

Och sedan ritar vi upp våra mätpunkter med matplotlib.pyplot.plot()

In [ ]:
plt.plot(Umeas, Imeas)

Vi kan anpassa utseendet av kurvan med olika parameter/attribut

In [ ]:
fig, ax = plt.subplots()
ax.plot(
    Umeas, Imeas, 
    color="blue", 
    linestyle=":", 
    linewidth=2, 
    marker="o", 
    markeredgecolor="blue", 
    markerfacecolor="red", 
    markersize=15
)

In [ ]:
fig, ax = plt.subplots()
ax.set(xlim=[-2,2], ylim=[-8,8])
ax.plot(
    Umeas, Imeas, 
    color="blue", 
    linestyle=":", 
    linewidth=2, 
    marker="o", 
    markeredgecolor="blue", 
    markerfacecolor="red", 
    markersize=15
)

Varifrån kommer den dubbla linjen?

Våra mätpunkter är osorterade - först kommer värden med positiva spänningar, sedan hoppar listan från den högsta positiva spänningen till den första negativa punkten.

Och Python bara drar en linje från punkt till punkt här.

Det är därför bättre att sortera mätpunkterna - inte för hand, fast med Python.

In [ ]:
data = list( zip( Umeas, Imeas ) )
sortdata = sorted(data)

x = [row[0] for row in sortdata]
y = [row[1] for row in sortdata]

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y, "b+:")

Mätpunkterna kommer från ett experiment med en spänningskälla, ett motstånd och en amperemeter. 

Därför borde Ohms lag gäller $I = U/R$ och för att hantera systematiska fel lägger vi till en konstant $I_{offs}$.

Låt oss se hur kurvan skulle se ut för $R = 300\,\text{Ω}$.

In [ ]:
def Iteor(U, R, Ioffs):
    return U / R * 1000 + Ioffs

calc_x = np.arange(-12, 12, 0.01)
calc_y = Iteor(calc_x, 300, 0)

`calc_x = np.arange(-12, 12, 0.01)` skapar en lista med spänningsvärden mellan $-12\,\text{V}$ och $+12\,\text{V}$ med steg på $0.01\,\text{V}=10\,\text{mV}$

`calc_y = Iteor(calc_x, 300, 0)` skapar en lista med ett funktionsvärde $f(x)=mx+b$ för alla värden från listan `calc_x`

## Kurvanpassning med `scipy.optimize.curve_fit`

Funktionen `curve_fit()` använder en numerisk optimering för att hitta parametrarna i en given modellfunktion `f` som ser till att den teoretiska modellen passar bäst till mätvärden `xdata` och `ydata`.

Även om det finns en riktig optimal lösning så finns det dock inga garantier att `curve_fit()` kommer lyckas, algoritmen kan fastna på lokala extrema istället. Därför är det alltid viktigt att visuellt jämnföra den anpassade kurvan med de ursprungliga mätpunkterna.

In [ ]:
optimizedParameters, pcov = opt.curve_fit(f=Iteor, xdata=x, ydata=y, p0=[1, 0], method="trf", verbose=2)

Obs! De sista raderna här ger oss informationen över hur bra Python lyckades med anpassningen. 

`optimizedParameters` är en lista som i vårt fall innehåller värden för $m$ och $b$

In [ ]:
print("m = {:.3f} Ω    b = {:.3g} A".format(optimizedParameters[0],optimizedParameters[1]))
print("I = U/{:.3f} Ω {:+.3g} A".format(optimizedParameters[0],optimizedParameters[1]))

Hur bra blev kurvanpassningen?

In [ ]:
calc_y = Iteor(calc_x, *optimizedParameters)

fig, ax = plt.subplots()
ax.plot(x, y, color="red", linestyle="", linewidth=2, marker="o",
         markeredgecolor="blue", markerfacecolor="red", markersize=15)
ax.plot(calc_x, calc_y, "b-")